# Langchain Components Demo

The full documentation on Langchain can be found at:
https://python.langchain.com/v0.1/docs/get_started/quickstart/

## Getting Started
Install the packages with pip.
1. Download and install [Anaconda Navigator](https://www.anaconda.com/download/)
2. Launch Jupyter Notebook from Anaconda Navigator
3. Go to "New" -> "Terminal"
4. Run the following command in the terminal
5. Open this `demo.ipynb` in Jupyter Notebook

```bash
pip install -U langchain==0.2.0
pip install -U langchain-core==0.2.0
pip install -U langchain-community==0.2.0
pip install -U langchain-openai==0.1.7
pip install -U langchain-experimental==0.0.59
pip install -U wikipedia
pip install -U yfinance
pip install -U tabulate
pip install python-dotenv
pip install langchainhub
```

## 1. Prompt Template
Prompt templates are **predefined recipes for generating prompts** for language models such as OpenAI's GPT model.

A template may include:
- instructions, 
- few-shot examples, and 
- specific context and questions appropriate for a given task.

Documentation: [Prompt Templates](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/)

### 1.1 PromptTemplate
Use PromptTemplate to create a template for a string prompt.

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

### 1.2 PromptTemplate with `input variables` and `partial variables`

Like other methods, it can make sense to "partial" a prompt template - e.g. `pass in a subset of the required values`, as to create a new prompt template which expects only the remaining subset of values.



In [ ]:
prompt_template = PromptTemplate(
    template="Tell me a {adjective} joke about {content}.",
    input_variables=["adjective", "content"],
)
prompt_template.format(adjective="funny", content="chickens")

In [ ]:
prompt_template = PromptTemplate(
    template="Tell me a {adjective} joke about {content}.",
    input_variables=["content"],
    partial_variables={"adjective": "funny"},
)
prompt_template.format(adjective="hilarious", content="chickens")

### 1.3 ChatPromptTemplate
ChatPromptTemplate is a template to `chat models` and it contains a list of `chat messages` that can be used to interact with the model.
Each chat message is associated with content, and an additional parameter called `role`. 

For example, in the `OpenAI Chat Completions API`, a chat message can be associated with an `AI assistant`, a `human` or a `system` role.

Let's create a chat template for `OpenAI chat model`.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
messages

Let's invoke the chat model with the generated messages.

In [ ]:
import os
import dotenv

from langchain_openai import AzureChatOpenAI

# Load environment variables from .env
dotenv.load_dotenv()

# Initialize the OpenAI Chat model
model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

model.invoke(messages)

### 1.4 MessagesPlaceholder
`MessagesPlaceholder` gives you full control of what messages to be rendered during formatting. 

This can be useful when you are uncertain of:
- what **role** you should be using for your message prompt templates or ;
- when you wish to insert a **list of messages** during formatting

Let's create a simple chat prompt using `MessagesPlaceholder`:

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

human_prompt = "Summarize our conversation so far in {word_count} words."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name="some_conversation"), human_message_template]
)
chat_prompt

Let's format the `chat_prompt` with some conversation messages.

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

human_message = HumanMessage(content="What is the best way to learn programming?")
ai_message = AIMessage(
    content="""\
1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
"""
)

# Format the chat prompt by passing the conversation messages
messages = chat_prompt.format_prompt(
    some_conversation=[human_message, ai_message], word_count="10"
).to_messages()

messages


Let's invoke the Open AI chat model with the generated `messages`.

In [ ]:
model.invoke(messages)

## 2. Output Parsers
Output parsers are responsible for taking the output of an LLM and **transforming it to a more suitable format** e.g `.json`, `.csv`. 

This is very useful when you are using LLMs to generate any form of structured data.

Documentation: [Output Parsers](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/)

### 2.1 Simple JsonOutputParser
This is a simple output parser that converts the output of an LLM to a JSON object.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

# Simple Json Output Parser
parser = JsonOutputParser()

# Get format instructions for the prompt template.
format_instructions = parser.get_format_instructions()

# Define a prompt template with the parser's format instructions.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)
prompt


Let's invoke the model with the `prompt` and `user_query`.

In [ ]:
import json

# Initialize the OpenAI Chat model
model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    temperature=0.0
)

# A query intended to prompt a language model to populate the data structure.
user_query = "What car has a very good horse power?"

# Invoke the model with the prompt
# model.invoke(prompt.format(query=user_query))

content = model.invoke(prompt.format(query=user_query)).content

json_obj = json.loads(content)
json_obj


### 2.2 JsonOutputParser with Pydantic
This output parser allows users to specify an **arbitrary JSON schema** and query LLMs for outputs that **conform to that schema**.


In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field

# Define your desired data structure for JSON.
class Car(BaseModel):
    brand: str = Field(description="the brand of the car in string")
    model: str = Field(description="the model of the car in string")
    hp: int = Field(description="the horse power of the car in integer")


# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Car)

# Get format instructions for the prompt template.
format_instructions = parser.get_format_instructions()

# Define a prompt template with the parser's format instructions.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)
prompt

Let's invoke the model with the `prompt` and `user_query`.

In [ ]:
# A query intended to prompt a language model to populate the data structure.
user_query = "What car has a very good horse power?"

# Invoke the model with the prompt
# model.invoke(prompt.format(query=user_query))

content = model.invoke(
    prompt.format(query=user_query)
).content

json_obj = json.loads(content)
json_obj

### 2.3 CSVOutputParser (Exercise)
Can you create a prompt template that output a `CSV` list of 5 `{subject}`?

E.g. List 5 `car brands`; List 5 `programming languages`;

Reference: [CommaSeparatedListOutputParser](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/csv/)


In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# Your code here

## 3. Chaining with LLMChain & LangChain Expression Language (LCEL)

A **Chain** refers to a sequence of steps or operations that are executed in a specific order to accomplish a task using LLMs and other computational components. 

Chains are a core concept in LangChain, allowing developers to build complex workflows that involve multiple interactions with:
- prompts,
- models,
- arbitrary functions,
- or even other chains

The primary supported way to do this is with `LCEL`.

Documentation: [Chaining](https://python.langchain.com/v0.1/docs/modules/chains/)



### 3.1 Simple Chaining with LLMChain


In [ ]:
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("List 5 {subject}.")

parser = StrOutputParser()

chain = LLMChain(
    llm=model, 
    prompt=prompt,
    output_parser=parser,
)

chain.invoke({"subject": subject})

### 3.1 Simple Chaining with LCEL
We can use LangChain runnables to chain together prompts, models, and parsers.

LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together.

LCEL makes it easy to build complex chains from basic components, and supports out of the box functionality such as streaming, parallelism, and logging.

Documentation: [LCEL](https://python.langchain.com/v0.1/docs/expression_language/)

In [ ]:
# Chaining different components using LCEL pipe operator
chain = prompt | model | parser

chain.invoke({"subject":subject})

### 3.2 Combined Chaining with LCEL (Sequential)
We can even combine this chain with more runnables to create another chain.

In [ ]:
# Define a second prompt template
analysis_prompt = ChatPromptTemplate.from_template("How do you classify these brands in terms of quality and pricing? {brands}")

combined_chain = {"brands": chain} | analysis_prompt | model | StrOutputParser()

combined_chain.invoke({"subject": subject})


### 3.3 Parrallel Chaining with LCEL
RunnableParallel (aka. RunnableMap) makes it easy to execute multiple Runnables in parallel, and to return the output of these Runnables as a map.

In [ ]:
from langchain_core.runnables import RunnableParallel

joke_prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
poem_prompt = ChatPromptTemplate.from_template("write a 2-line poem about {topic}")

joke_chain = joke_prompt | model | StrOutputParser()
poem_chain = poem_prompt | model | StrOutputParser()

map_chain = RunnableParallel(
    joke=joke_chain, 
    poem=poem_chain
)
map_chain.invoke({"topic": "cat"})


### 3.4 Using custom functions within Chain
You can also use arbitrary functions in the pipeline.

In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnableLambda

def get_length(text):
    return len(text)

def get_multiple_length(_dict):
    return len(_dict["text1"]) * len(_dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(get_length),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}| RunnableLambda(get_multiple_length),
    }
    | prompt
    | model
    | StrOutputParser() 
)

chain.invoke({"foo": "hi", "bar": "world"})

## 4. Memory
Most LLM applications have a conversational interface. 

An essential component of a conversation is being able to refer to information introduced earlier in the conversation. 

At bare minimum, a conversational system should be able to access some window of past messages directly. 

Documentation: [Memory](https://python.langchain.com/v0.1/docs/modules/memory/)

### 4.1 In-Memory with LCEL
The `RunnableWithMessageHistory` lets us add message history to certain types of chains. It wraps another `Runnable` and manages the chat message history for it.

Below we show a simple example in which the chat history lives in memory, in this case via a global Python dict.

Documentation: [Memory LCEL](https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/)

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        (
            "human", 
            "{input}"
        ),
    ]
)

runnable = prompt | model

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        # Create a new chat message history
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    # the key to be treated as the latest input message
    input_messages_key="input",
    # the key to add historical messages to
    history_messages_key="history",
)

In [ ]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "abc123"}},
)


Recalling the last message

In [ ]:
# Remembers
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "abc123"}},
)

What if there isn't a message history to refer to?

In [ ]:
# New session_id --> does not remember.
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "def234"}},
)

## 5. Tools and Toolkits
Tools are interfaces that an agent, chain, or LLM can use to interact with the world. They combine a few things:

- The name of the tool
- A description of what the tool is
- JSON schema of what the inputs to the tool are
- The function to call
- Whether the result of a tool should be returned directly to the user

It is useful to have all this information because this information can be used to build action-taking systems! The name, description, and JSON schema can be used to prompt the LLM so it knows how to specify what action to take, and then the function to call is equivalent to taking that action.

Documentation: [Toolkits](https://python.langchain.com/v0.1/docs/modules/tools/)

### 5.1 Default Tools (Wikipedia Query Tool)
The Wikipedia Query tool allows you to query Wikipedia for information.


In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

Important information for LLM to recoginise and make use of the tool.

In [ ]:
tool.name

In [ ]:
tool.description

In [ ]:
tool.args

Let's run the tool with a query!

In [ ]:
tool.run({"query": "langchain"})

### 5.2 Yahoo Finance News Tool (Exercise)
The Yahoo Finance News tool allows you to query Yahoo Finance for news articles.

Hint: use `tool.name`, `tool.description`, and `tool.args` to understand the tool.

In [ ]:
from langchain_community.tools import YahooFinanceNewsTool

# Your code here

### 5.3 Toolkits
Toolkits are collections of tools that are designed to be used together for specific tasks. They have convenient loading methods. 

For a complete list of available ready-made toolkits, visit [Integrations](https://python.langchain.com/v0.1/docs/integrations/toolkits/).

We will demonstrate this using `Agents`.

## 6. Agents
The core idea of agents is to use a language model to choose a sequence of actions to take. 

In chains, a sequence of actions is **hardcoded** (in code). 

In agents, a language model is used as a **reasoning engine** to determine which actions to take and in which order.

Documentation: [Agents](https://python.langchain.com/v0.2/docs/integrations/toolkits/)

### 6.1 Agent with CSV Tooklits
This notebook shows how to use `agents` to interact with data in `CSV format`. 

It is mostly optimized for question answering.

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent

# A zero shot agent that does a reasoning step before acting.
agent = create_csv_agent(
    model,
    "titanic.csv",
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
agent.invoke("What is the average age of the passengers?")

Alternative `agent_type`:

In [ ]:
# An agent optimized for using open AI functions.
agent = create_csv_agent(
    model,
    "titanic.csv",
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)
agent.invoke("What is the average age of the passengers?")

In [ ]:
agent.invoke("how many people have more than 3 siblings")

### 6.2 Agent with Python Toolkit
We can design the agent to write and execute Python code to answer a question.

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_experimental.tools import PythonREPLTool


tools = [PythonREPLTool()]
instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
# Pulling a predefined prompt template from the Langchain hub. 
# This template is designed for OpenAI's language models
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
agent = create_openai_functions_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What is the 10th fibonacci number?"})


### 7. Huggingingface Models
The [Hugging Face](https://huggingface.co/docs/hub/index) Hub is a platform with over 120k models, 20k datasets, and 50k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

Documentation: [Huggingface Endpoints](https://python.langchain.com/v0.1/docs/integrations/llms/huggingface_endpoint/)
